In [3]:
from Extractor_Code.json_extractor import JSONExtractor
from Extractor_Code.database_connector import DatabaseConnector
from Extractor_Code.api_extractor import APIExtractor


In [4]:
from dotenv import load_dotenv
import yaml
import os

In [6]:
# adjust this path to where your .env actually lives
load_dotenv(r".env")

config_path = r"config.yaml"
with open(config_path) as f:
    raw = f.read()

In [7]:
# expands ${DB_HOST}, ${DB_USER}, etc. using os.environ
expanded = os.path.expandvars(raw)
config = yaml.safe_load(expanded)

# sanity check: should be a dict-of-dicts
print("CONFIG:", config)
print("TYPE of config['database']:", type(config['database']))

CONFIG: {'database': {'host': 'localhost', 'database': 'advancedsql', 'user': 'postgres', 'password': 'Harshana123', 'port': 5432}, 's3': {'bucket_name': 'fde-bucket-my', 'region': 'us-east-1', 'files': {'JSON/products.json': 'lnd_products_json', 'JSON/sales.json': 'lnd_sales_json', 'CSV/customers.csv': 'lnd_customers_csv', 'CSV/products.csv': 'lnd_products_csv', 'CSV/sales.csv': 'lnd_sales_csv'}}, 'api': {'endpoints': {'https://dummyjson.com/products': 'lnd_products_api', 'https://dummyjson.com/users': 'lnd_users_api'}}, 'schemas': {'landing_schema': 'landing', 'staging_schema': 'staging', 'transform_schema': 'transform', 'target_schema': 'target'}, 'entities': {'products': {'staging_view': 'stg_products', 'temp_table': 'tmp_products', 'target_table': 'dim_products'}, 'users': {'staging_view': 'stg_users', 'temp_table': 'tmp_users', 'target_table': 'dim_users'}, 'sales': {'staging_view': 'stg_sales', 'temp_table': 'tmp_sales', 'target_table': 'fact_sales'}}}
TYPE of config['database']

In [8]:
# 3) instantiate your connector with that dict
db = DatabaseConnector(config)

In [9]:
conn = db.get_connection()
cursor = conn.cursor()


In [10]:
table_name = 'lnd_products_json'


In [11]:
cursor.execute("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'Landing' AND table_name = %s
""", (table_name,))
columns = [row[0] for row in cursor.fetchall()]
has_file_name = 'file_name' in columns
print(f"Table '{table_name}' has 'file_name' column: {has_file_name}")

Table 'lnd_products_json' has 'file_name' column: False


### API


In [12]:
config['api']['endpoints']

{'https://dummyjson.com/products': 'lnd_products_api',
 'https://dummyjson.com/users': 'lnd_users_api'}

In [13]:
json_extractor = JSONExtractor(db)

In [14]:
api_extractor = APIExtractor(config,json_extractor)

In [15]:
api_extractor.extract_endpoint('https://dummyjson.com/products', 'lnd_products_api')

INFO:Extractor_Code.json_extractor:Loaded JSON data to lnd_products_api
INFO:Extractor_Code.api_extractor:Loaded API data from https://dummyjson.com/products to lnd_products_api


In [16]:
api_extractor.extract_all()

INFO:Extractor_Code.api_extractor:Processing https://dummyjson.com/products
INFO:Extractor_Code.json_extractor:Loaded JSON data to lnd_products_api
INFO:Extractor_Code.api_extractor:Loaded API data from https://dummyjson.com/products to lnd_products_api
INFO:Extractor_Code.api_extractor:Processing https://dummyjson.com/users
INFO:Extractor_Code.json_extractor:Loaded JSON data to lnd_users_api
INFO:Extractor_Code.api_extractor:Loaded API data from https://dummyjson.com/users to lnd_users_api
